<a href="https://colab.research.google.com/github/robinpats182/Computer-Vision-CNN-Essentials/blob/main/CNN_Inception_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class InceptionModule(layers.Layer):
    def __init__(self, filters):
        super(InceptionModule, self).__init__()
        self.conv1x1 = layers.Conv2D(filters[0], kernel_size=(1, 1), activation='relu')
        self.conv3x3_reduce = layers.Conv2D(filters[1], kernel_size=(1, 1), activation='relu')
        self.conv3x3 = layers.Conv2D(filters[2], kernel_size=(3, 3), padding='same', activation='relu')
        self.conv5x5_reduce = layers.Conv2D(filters[3], kernel_size=(1, 1), activation='relu')
        self.conv5x5 = layers.Conv2D(filters[4], kernel_size=(5, 5), padding='same', activation='relu')
        self.pool = layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')
        self.conv1x1_proj = layers.Conv2D(filters[5], kernel_size=(1, 1), activation='relu')

    def call(self, inputs):
        branch1 = self.conv1x1(inputs)

        branch2 = self.conv3x3_reduce(inputs)
        branch2 = self.conv3x3(branch2)

        branch3 = self.conv5x5_reduce(inputs)
        branch3 = self.conv5x5(branch3)

        branch4 = self.pool(inputs)
        branch4 = self.conv1x1_proj(branch4)

        output = tf.concat([branch1, branch2, branch3, branch4], axis=-1)
        return output

class InceptionModel(Model):
    def __init__(self):
        super(InceptionModel, self).__init__()
        self.conv = layers.Conv2D(64, kernel_size=(7, 7), strides=(1, 1), padding='same', activation='relu')
        self.pool = layers.MaxPooling2D(pool_size=(2, 2))
        self.inception1 = InceptionModule([64, 96, 128, 16, 32, 32])
        self.inception2 = InceptionModule([128, 128, 192, 32, 96, 64])
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.pool(x)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Create an instance of the InceptionModel
model = InceptionModel()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=12, validation_data=(x_test, y_test))


Epoch 1/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - accuracy: 0.7937 - loss: 0.5634 - val_accuracy: 0.8816 - val_loss: 0.3165
Epoch 2/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.9017 - loss: 0.2688 - val_accuracy: 0.9065 - val_loss: 0.2624
Epoch 3/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.9195 - loss: 0.2187 - val_accuracy: 0.9085 - val_loss: 0.2519
Epoch 4/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9317 - loss: 0.1893 - val_accuracy: 0.9135 - val_loss: 0.2462
Epoch 5/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.9409 - loss: 0.1578 - val_accuracy: 0.9174 - val_loss: 0.2352
Epoch 6/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 35ms/step - accuracy: 0.9516 - loss: 0.1340 - val_accuracy: 0.9190 - val_loss: 0.2370
Epoch 7/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 35ms/step - accuracy: 0.9562 - loss: 0.1166 - val_accuracy: 0.9218 - val_loss: 0.2443
Epoch 8/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.9688 - loss: 0.0869 - 

[0.36178600788116455, 0.9214000105857849]

In [ ]:
# Evaluate the model
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9207 - loss: 0.3542


[0.36178600788116455, 0.9214000105857849]